In [1]:
"""
While doing '2024-06-03 Inhalte nach Schultyp' 
I noticed some differences between the metadata query and another way of getting entities.
So, I decided to investigate it further and came to some interesting results.
If you want a TD;DR, scroll down
"""

%run 'get_all_entities_from_taxonomy_tree.ipynb'

In [2]:
def naive_entities_query(instance_id=None):
    return queryone(
        """
    SELECT COUNT(*) FROM entity
    JOIN uuid ON uuid.id = entity.id
    JOIN type ON type_id = type.id
    WHERE trashed = 0
        AND (%s is NULL OR entity.instance_id = %s)
        AND type.name IN ("applet", "article", "course", "text-exercise",
                          "text-exercise-group", "video")
    """,
        [instance_id, instance_id],
    )[0]


all_naive = naive_entities_query()
all_naive_german = naive_entities_query(1)

print(
    f"""
This is a simple entity query.
Total not trashed entities of types supported by metadata query: {all_naive}
Total such German ones: {all_naive_german}
"""
)


This is a simple entity query.
Total not trashed entities of types supported by metadata query: 13964
Total such German ones: 12266



In [3]:
def entities_with_current_revision(instance_id=None):
    return queryone(
        """
    SELECT COUNT(*) FROM entity
    JOIN uuid ON uuid.id = entity.id
    JOIN entity_revision ON entity_revision.id = current_revision_id
    JOIN type ON type_id = type.id
    WHERE trashed = 0
        AND (%s is NULL OR entity.instance_id = %s)
        AND type.name IN ("applet", "article", "course", "text-exercise",
                          "text-exercise-group", "video")
    """,
        [instance_id, instance_id],
    )[0]


all_current = entities_with_current_revision()
all_current_german = entities_with_current_revision(1)
difference_all_current = all_naive - all_current

print(
    f"""
This is a query for live entities (again just the supported types by metadata query).
Total: {all_current}
German: {all_current_german}

As you can see, there is a lot of entities without any revision
All instances: {difference_all_current}
German: {all_naive_german - all_current_german}
"""
)


This is a query for live entities (again just the supported types by metadata query).
Total: 11072
German: 9701

As you can see, there is a lot of entities without any revision
All instances: 2892
German: 2565



In [4]:
def entities_with_valid_license(instance_id=None):
    return queryall(
        """
    SELECT entity.id FROM entity
    JOIN uuid ON uuid.id = entity.id
    JOIN type ON type_id = type.id
    JOIN entity_revision on current_revision_id = entity_revision.id
    JOIN license ON license.id = entity.license_id
    WHERE trashed = 0
        AND (%s is NULL OR entity.instance_id = %s)
        AND type.name IN ("applet", "article", "course", "text-exercise",
                          "text-exercise-group", "video")
    """,
        [instance_id, instance_id],
    )


all_license = entities_with_valid_license()
german_license = entities_with_valid_license(1)

# Different from the previous one, we have to see how many rows are there, because we groupby the count
number_all_license = len(all_license)
number_german_license = len(german_license)
difference_all_license = number_all_license - all_current


difference_all_license = all_current - number_all_license

print(
    f"""
If we apply the license filter we have the following result:
Total with license: {number_all_license}
German with license: {number_german_license}

Now, a lot of entities are excluded, because they haven't any license that are valid in the database.
All excluded: {difference_all_license}
German excluded: {all_current_german - number_german_license}
"""
)


If we apply the license filter we have the following result:
Total with license: 9903
German with license: 8532

Now, a lot of entities are excluded, because they haven't any license that are valid in the database.
All excluded: 1169
German excluded: 1169



In [5]:
# The function get_entities_with_taxonomy() is in the get_all_entities_from_taxonomy_tree.ipynb
all_with_taxonomy = get_entities_with_taxonomy()
german_with_taxonomy = get_entities_with_taxonomy(1)

# Different from the previous one, we have to see how many rows are there, because we groupby the count
number_all_with_taxonomy = len(all_with_taxonomy)
number_german_with_taxonomy = len(german_with_taxonomy)
difference_with_taxonomy = number_all_license - number_all_with_taxonomy

print(
    f"""
And if we go on filter the ones without taxonomies we get
Total with taxonomy: {number_all_with_taxonomy}
German: {number_german_with_taxonomy}

It means we have {difference_with_taxonomy} entities without any taxonomy. 
In German instance it is {number_german_license - number_german_with_taxonomy}
"""
)


And if we go on filter the ones without taxonomies we get
Total with taxonomy: 9616
German: 8258

It means we have 287 entities without any taxonomy. 
In German instance it is 274



In [6]:
"""
Metadata query as it is BEFORE merging the entity_revision_field into entity_revision
Current query in production with some changes
See https://github.com/serlo/api.serlo.org/blob/production/packages/server/src/schema/metadata/resolvers.ts
I removed the limit and the pagination and added a parameter for querying the subject, 
in order to know how many are in German mathematics instance.


NOTE: the merging of the entity_revision_field may occur in productionsoon,
so we could adapt and rerun this notebooke then
"""
from datetime import date

display(Markdown(f"### Last Update: {date.today()}"))


def metadata_query(subdomain=None, subject_id=None):
    return queryall(
        """
       WITH RECURSIVE subject_mapping AS (
            SELECT
                subject.id AS term_taxonomy_id,
                subject.id AS subject_id,
                root.id AS root_id
            FROM term_taxonomy root
            JOIN term_taxonomy subject ON subject.parent_id = root.id
            WHERE root.parent_id IS NULL
            OR root.id IN (106081, 146728)

            UNION

            SELECT
                child.id,
                subject_mapping.subject_id,
                subject_mapping.root_id
            FROM term_taxonomy child
            JOIN subject_mapping ON subject_mapping.term_taxonomy_id = child.parent_id
            -- "Fächer im Aufbau" taxonomy is on the level of normal Serlo subjects, therefore we need a level below it.
            -- "Partner" taxonomy is below the subject "Mathematik", but we only want the entities with the specific partner as the subject.
            WHERE child.parent_id NOT IN (87993, 106081, 146728)
                -- Exclude content under "Baustelle", "Community", "Zum Testen" and "Testbereich" taxonomies
                AND child.id NOT IN (75211, 105140, 107772, 135390, 25107, 106082)
        )
        SELECT
            entity.id,
            JSON_ARRAYAGG(subject_mapping.subject_id) AS subjectIds,
            type.name AS resourceType,
            MIN(field_title.value) AS title,
            MIN(field_description.value) AS description,
            entity.date AS dateCreated,
            entity_revision.date AS dateModified,
            entity.current_revision_id AS currentRevisionId,
            license.url AS licenseUrl,
            license.original_author_url as originalAuthorUrl,
            instance.subdomain AS instance,
            JSON_ARRAYAGG(term_taxonomy.id) AS taxonomyTermIds,
            JSON_OBJECTAGG(term_taxonomy.id, term.name) AS termNames,
            JSON_OBJECTAGG(user.id, user.username) AS authors,
            JSON_OBJECTAGG(all_revisions_of_entity.id, user.id) AS authorEdits
        FROM entity
        JOIN uuid ON uuid.id = entity.id
        JOIN instance ON entity.instance_id = instance.id
        JOIN type on entity.type_id = type.id
        JOIN license on license.id = entity.license_id
        JOIN entity_revision ON entity.current_revision_id = entity_revision.id
        LEFT JOIN entity_revision_field field_title on
            field_title.entity_revision_id = entity_revision.id AND
            field_title.field = "title"
        LEFT JOIN entity_revision_field field_description on
            field_description.entity_revision_id = entity_revision.id AND
            field_description.field = "meta_description"
        JOIN term_taxonomy_entity on term_taxonomy_entity.entity_id = entity.id
        JOIN term_taxonomy on term_taxonomy_entity.term_taxonomy_id = term_taxonomy.id
        JOIN term on term_taxonomy.term_id = term.id
        JOIN entity_revision all_revisions_of_entity ON all_revisions_of_entity.repository_id = entity.id
        JOIN user ON all_revisions_of_entity.author_id = user.id
        JOIN subject_mapping on subject_mapping.term_taxonomy_id = term_taxonomy_entity.term_taxonomy_id
        WHERE (%s is NULL OR instance.subdomain = %s)
            AND uuid.trashed = 0
            AND type.name IN ("applet", "article", "course", "text-exercise",
                              "text-exercise-group", "video")
            AND (%s is NULL OR subject_mapping.subject_id = %s)
            AND NOT subject_mapping.subject_id = 146728
        GROUP BY entity.id
        ORDER BY entity.id
        """,
        [subdomain, subdomain, subject_id, subject_id],
    )


all_metadata = len(metadata_query())
all_metadata_german = len(metadata_query("de"))
print(
    f"""
Now lets see the metadata query 
All metadata entities {all_metadata}
German: {all_metadata_german}
"""
)

### Last Update: 2024-06-13


Now lets see the metadata query 
All metadata entities 8733
German: 7375



In [7]:
def metadata_query_no_filter(subdomain=None, subject_id=None):
    return queryall(
        """
        WITH RECURSIVE subject_mapping AS (
            SELECT
                subject.id AS term_taxonomy_id,
                subject.id AS subject_id,
                root.id AS root_id
            FROM term_taxonomy root
            JOIN term_taxonomy subject ON subject.parent_id = root.id
            WHERE root.parent_id IS NULL

            UNION

            SELECT
                child.id,
                subject_mapping.subject_id,
                subject_mapping.root_id
            FROM term_taxonomy child
            JOIN subject_mapping ON subject_mapping.term_taxonomy_id = child.parent_id
        )
        SELECT
           entity_id
        FROM entity
        JOIN uuid ON uuid.id = entity.id
        JOIN instance ON entity.instance_id = instance.id
        JOIN type on entity.type_id = type.id
        JOIN license on license.id = entity.license_id
        JOIN entity_revision ON entity.current_revision_id = entity_revision.id
        LEFT JOIN entity_revision_field field_title on
            field_title.entity_revision_id = entity_revision.id AND
            field_title.field = "title"
        LEFT JOIN entity_revision_field field_description on
            field_description.entity_revision_id = entity_revision.id AND
            field_description.field = "meta_description"
        JOIN term_taxonomy_entity on term_taxonomy_entity.entity_id = entity.id
        JOIN term_taxonomy on term_taxonomy_entity.term_taxonomy_id = term_taxonomy.id
        JOIN term on term_taxonomy.term_id = term.id
        JOIN entity_revision all_revisions_of_entity ON all_revisions_of_entity.repository_id = entity.id
        JOIN user ON all_revisions_of_entity.author_id = user.id
        JOIN subject_mapping on subject_mapping.term_taxonomy_id = term_taxonomy_entity.term_taxonomy_id
        WHERE (%s is NULL OR instance.subdomain = %s)
            AND uuid.trashed = 0
            AND type.name IN ("applet", "article", "course", "text-exercise",
                              "text-exercise-group", "video")
            -- added to check how much are under Math
            AND (%s is NULL OR subject_mapping.subject_id = %s)
        GROUP BY entity.id
        ORDER BY entity.id
        """,
        [subdomain, subdomain, subject_id, subject_id],
    )


metadata_no_filter = len(metadata_query_no_filter())
german_no_filter = len(metadata_query_no_filter("de"))

print(
    f"""
Ok, the difference may be caused by the fact that we filter out entities under 
Testbereich etc.
If we query metadata without any filter, we get:

Total: {metadata_no_filter}
German: {german_no_filter}

Alright, are metadata query and query filtering out entities without taxonomy equal? {number_all_with_taxonomy == metadata_no_filter}
"""
)


Ok, the difference may be caused by the fact that we filter out entities under 
Testbereich etc.
If we query metadata without any filter, we get:

Total: 9616
German: 8258

Alright, are metadata query and query filtering out entities without taxonomy equal? True



In [8]:
# explanation
%run 'get_all_entities_from_taxonomy_tree.ipynb'

In [9]:
german_tree = len(get_all_entities_from_taxonomy_tree([3]))
print(
    f"""
Now, querying from the taxonomy tree (it means, we start from German root 
taxonomy and go down to all entities) is somehow getting less than querying from the
instance 
From taxonomy tree {german_tree}
Difference: {german_no_filter - german_tree}

This happens because there are some taxonomies similar as German Root, i.e. they don't have parent and
are in the German instance.

Anyway, I think it is safe to keep using get_all_entities_from_taxonomy_tree in the notebook for Schultypen
"""
)


Now, querying from the taxonomy tree (it means, we start from German root 
taxonomy and go down to all entities) is somehow getting less than querying from the
instance 
From taxonomy tree 8220
Difference: 38

This happens because there are some taxonomies similar as German Root, i.e. they don't have parent and
are in the German instance.

Anyway, I think it is safe to keep using get_all_entities_from_taxonomy_tree in the notebook for Schultypen



In [10]:
# explanation
%run 'get_all_entities_from_taxonomy_tree.ipynb'

In [11]:
math_metadata = len(metadata_query_no_filter("de", 5))
math_from_tree = len(get_all_entities_from_taxonomy_tree([5]))

print(
    f"""
At last, regarding German mathematics we get the following results

using the (modified) metadata query: {math_metadata}
and getting the entities from taxonomy tree: {math_from_tree}

Difference: {math_metadata - math_from_tree}

I can't really explain why there is this difference, but I think anyway it is safe to ignore it.
"""
)


At last, regarding German mathematics we get the following results

using the (modified) metadata query: 5557
and getting the entities from taxonomy tree: 5525

Difference: 32

I can't really explain why there is this difference, but I think anyway it is safe to ignore it.



In [12]:
all_license_elements = [row[0] for row in all_license]
all_with_taxonomy_elements = [row[0] for row in all_with_taxonomy]

without_taxonomy = set(all_license_elements) - set(all_with_taxonomy_elements)

display(
    Markdown(
        f"""
# Results

### There are a lot of entities ({difference_all_current}) without any current revisions 
Of course, some of them are just waiting for review, but I think we just have a bad use of the platforma,
that the content unit could take care of.
### Metadata Query is excluding a lot of entities ({difference_all_license}) which have license that isn't found in the database 
Since we changed the way of handling the license, they are probably valid.  
_Action_: Stop filtering the license in the metadata query and handle it in another way.
### There are some entities ({difference_with_taxonomy}) without any taxomomy. These are they: 
"""
    )
)
for t in sorted(without_taxonomy):
    display(Markdown(f"[https://serlo.org/{t}](https://serlo.org/{t})"))


# Results

### There are a lot of entities (2892) without any current revisions 
Of course, some of them are just waiting for review, but I think we just have a bad use of the platforma,
that the content unit could take care of.
### Metadata Query is excluding a lot of entities (1169) which have license that isn't found in the database 
Since we changed the way of handling the license, they are probably valid.  
_Action_: Stop filtering the license in the metadata query and handle it in another way.
### There are some entities (287) without any taxomomy. These are they: 


[https://serlo.org/21071](https://serlo.org/21071)

[https://serlo.org/21073](https://serlo.org/21073)

[https://serlo.org/22676](https://serlo.org/22676)

[https://serlo.org/22685](https://serlo.org/22685)

...

[https://serlo.org/221326](https://serlo.org/221326)

[https://serlo.org/222411](https://serlo.org/222411)

[https://serlo.org/223423](https://serlo.org/223423)

[https://serlo.org/226107](https://serlo.org/226107)

[https://serlo.org/251303](https://serlo.org/251303)